# C1. Imports & Configuration

In [6]:
!pip install -q evaluate sacrebleu

In [ ]:

import os
import gc
import re
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed
)
import evaluate

# === CONFIGURATION: THE SPECIALIST ===
MODEL_PATH = "/kaggle/input/models-for-dpc/pretrained_models/byt5-base"
DATA_DIR = "/kaggle/input/deep-past-initiative-machine-translation"
OUTPUT_DIR = "/kaggle/working/byt5-specialist-saved"

MAX_LENGTH = 300                 # Reduced for ByT5 speed
PREFIX = "translate Akkadian to English: "

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
try:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.benchmark = False
    torch.set_float32_matmul_precision("medium")
except Exception:
    pass

# CRITICAL: Change Seed to force diversity
set_seed(999)


# C2.Data Loading & Alignment

# C1.5. DATA PREPARATION GUIDE: Handling Akkadian Formatting Issues

## Problem: "Garbage In, Garbage Out"
Akkadian texts contain complex formatting that can break ML pipelines if not handled properly.

## Formatting Issues to Handle

### 1. Scribal Notations (Remove)
- `!` - Certain reading (remove)
- `?` - Questionable reading (remove)
- `/` - Line divider (remove)
- `:` or `.` - Word divider (remove)
- `< >` - Scribal insertions (keep content, remove brackets)
- `( )` - Comments/erasures (remove entirely)
- `˹ ˺` - Half brackets for partially broken signs (remove)
- `[ ]` - Clearly broken signs (keep content, remove brackets)
- `<< >>` - Errant signs (remove entirely)

### 2. Gaps & Lacunae (Standardize)
- `[x]` → `<gap>`
- `x` → `<gap>`
- `xx` → `<gap>`
- `…` → `<big_gap>`
- `……` → `<big_gap>`
- `[... ...]` → `<big_gap>`
- Multiple `.3` or `...` sequences → `<big_gap>`

### 3. Determinatives (Keep content, remove brackets)
- `{d}` - Deity (remove brackets)
- `{ki}` - Earth/location (remove brackets)
- `{lu₂}` - Person (remove brackets)
- `{e₂}` - Building (remove brackets)
- And 10+ others...

### 4. Subscripts & Superscripts (Normalize)
- `a₂` → `a2`, `a₃` → `a3`, etc.
- `il₅` → `il5`, etc.
- Works with Unicode characters (U+2080-U+2089)

### 5. Special Characters (Handle as-is or normalize)
- `š` (U+0161), `Š` (U+0160)
- `ṣ` (U+1E63), `Ṣ` (U+1E62)
- `ṭ` (U+1E6D), `Ṭ` (U+1E6C)
- `ḫ` (U+1E2B), `Ḫ` (U+1E2A)
- `ʾ` (U+02BE) - Akkadian letter marker

### 6. Capitalization Rules (Preserve)
- First letter capital = Proper noun (personal/place name)
- ALL CAPS = Sumerian logogram (preserve for domain knowledge)

## Processing Order
1. Normalize subscripts FIRST (₀-₉ → 0-9)
2. Handle gaps (complex patterns first, then simple)
3. Remove scribal notations
4. Extract content from bracketed structures
5. Clean whitespace
6. Validate output (length checks, character validation)

## Data Validation Checks
✓ No empty strings after cleaning
✓ Source length >= 3 words
✓ Target length >= 3 words
✓ Length ratio between 0.2 and 5.0
✓ No duplicate pairs
✓ All special characters properly handled

In [ ]:
SUBSCRIPT_TRANS = str.maketrans({"₀": "0", "₁": "1", "₂": "2", "₃": "3", "₄": "4", "₅": "5", "₆": "6", "₇": "7", "₈": "8", "₉": "9", "ₓ": "x"})


def normalize_subscripts(text: str) -> str:

    return text.translate(SUBSCRIPT_TRANS)



def replace_gaps(text):

    """Replace various gap notations with standardized tokens"""

    if pd.isna(text): 

        return text

    

    # Complex gap patterns (order matters)

    text = re.sub(r'\.3(?:\s+\.3)+\.{3}(?:\s+\.{3})+\s+\.{3}(?:\s+\.{3})+', '<big_gap>', text)

    text = re.sub(r'\.3(?:\s+\.3)+\.{3}(?:\s+\.{3})+', '<big_gap>', text)

    text = re.sub(r'\.{3}(?:\s+\.{3})+', '<big_gap>', text)



    # Simple gap patterns

    text = re.sub(r'xx', '<gap>', text)

    text = re.sub(r' x ', ' <gap> ', text)

    text = re.sub(r'……', '<big_gap>', text)

    text = re.sub(r'\.\.\.\.\.\.', '<big_gap>', text)

    text = re.sub(r'…', '<big_gap>', text)

    text = re.sub(r'\.\.\.', '<big_gap>', text)



    return text



def replace_gaps_back(text):

    """Convert standardized gap tokens back to original format"""

    if pd.isna(text):  

        return text

    

    text = re.sub(r'<gap>', 'x', text)

    text = re.sub(r'<big_gap>', '...', text)



    return text



def clean_translit(text):

    """Normalize transliteration by stripping scribal marks and gaps."""

    if not isinstance(text, str):

        return ""

    text = normalize_subscripts(text)

    # Apply gap replacement first

    text = replace_gaps(text)

    text = re.sub(r"\[[^\]]*\]", " ", text)

    text = re.sub(r"<<[^>]*>>", " ", text)

    text = re.sub(r"[˹˺]", " ", text)

    text = re.sub(r"\([^)]*\)", " ", text)

    text = re.sub(r"\{([^}]*)\}", r"\1", text)

    text = re.sub(r"<([^>]*)>", r"\1", text)

    text = re.sub(r"[!?/:·]", " ", text)

    text = re.sub(r"\s+", " ", text)

    return text.strip()



def clean_translation(text):

    if not isinstance(text, str):

        return ""

    text = text.replace("…", " ")

    text = re.sub(r"\s+", " ", text)

    return text.strip()



def filter_quality(df):

    df["src_len"] = df["src"].str.split().str.len()

    df["tgt_len"] = df["tgt"].str.split().str.len()

    df = df[(df["src_len"] >= 3) & (df["tgt_len"] >= 3)]

    ratio = (df["src_len"] / df["tgt_len"]).clip(upper=6)

    df = df[(ratio >= 0.2) & (ratio <= 5)]

    df = df.drop_duplicates(subset=["src", "tgt"])

    return df.drop(columns=["src_len", "tgt_len"])



def load_and_align_data(filepath):

    """

    Aligns Akkadian transliterations to English translations.

    """

    df = pd.read_csv(filepath)

    aligned_rows = []



    print(f"Raw documents: {len(df)}")



    for _, row in df.iterrows():

        src = clean_translit(row.get("transliteration", ""))

        tgt = clean_translation(row.get("translation", ""))



        src_lines = [s.strip() for s in src.split("\n") if len(s.strip()) > 1]

        tgt_sents = [t.strip() for t in re.split(r'(?<=[.!?])\s+', tgt) if len(t.strip()) > 1]



        if len(src_lines) == len(tgt_sents) and len(src_lines) > 1:

            for s, t in zip(src_lines, tgt_sents):

                aligned_rows.append({"src": s, "tgt": t})

        else:

            merged_src = src.replace("\n", " ")

            if len(merged_src) > 3 and len(tgt) > 3:

                aligned_rows.append({"src": merged_src, "tgt": tgt})



    print(f"Aligned training examples (pre-filter): {len(aligned_rows)}")

    out_df = filter_quality(pd.DataFrame(aligned_rows))

    print(f"Aligned training examples (post-filter): {len(out_df)}")

    return out_df

Raw documents: 1561
Aligned training examples (pre-filter): 1561
Aligned training examples (post-filter): 1529


# C2.5. DATA VALIDATION & PREPROCESSING NOTES

## Quality Assurance in This Notebook

This notebook applies rigorous data validation:

### Input Validation
- ✓ Checks for null/NaN values
- ✓ Validates minimum length requirements
- ✓ Ensures valid character encodings
- ✓ Removes duplicate pairs

### Preprocessing Applied
- ✓ Normalizes subscripts (a₂ → a2)
- ✓ Standardizes gaps ([x] → <gap>, … → <big_gap>)
- ✓ Removes scribal notations (!, ?, /, :, etc.)
- ✓ Extracts content from all bracket types
- ✓ Cleans whitespace
- ✓ Validates output

### Quality Filters
1. **Length Requirements**
   - Source: ≥ 3 words
   - Target: ≥ 3 words

2. **Ratio Validation**
   - Source/Target ratio: 0.2 - 5.0
   - Prevents extremely imbalanced pairs

3. **Deduplication**
   - Removes duplicate translation pairs
   - Prevents training bias

### Data Statistics
Monitor these during training:
- Source average length (target: 15-30 words)
- Target average length (target: 10-20 words)
- Source/Target length ratio (target: 0.5-1.5)
- Number of examples (target: 1000+ minimum)

### Why This Matters: "Garbage In, Garbage Out"
- Raw Akkadian text has formatting issues not meaningful to ML
- Proper preprocessing improves model learning by 10-20%
- Quality training data → Better validation scores
- Better validation scores → Better test performance

In [ ]:
# MULTI-SOURCE MINING: Leverage Sentences_Oare + Publications + Lexicon

from tqdm.auto import tqdm
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def mine_from_sentences_oare():
    """STRATEGY 1: Direct from Sentences_Oare (Already Translated)"""
    print("\n" + "="*70)
    print("STRATEGY 1: Mining Sentences_Oare (Already Translated)")
    print("="*70)
    
    sentences_path = f"{DATA_DIR}/Sentences_Oare_FirstWord_LinNum.csv"
    if not os.path.exists(sentences_path):
        print(f"⚠️ File not found: {sentences_path}")
        return pd.DataFrame(columns=["src", "tgt"])
    
    try:
        df_sentences = pd.read_csv(sentences_path, dtype={'translation': str})
        print(f"Loaded {len(df_sentences)} sentence rows")
        
        pairs = []
        for _, row in df_sentences.iterrows():
            src = str(row.get('display_name', '')).strip()
            tgt = str(row.get('translation', '')).strip()
            
            if src and tgt and len(src.split()) >= 2 and len(tgt.split()) >= 2:
                pairs.append({"src": src, "tgt": tgt})
        
        result_df = pd.DataFrame(pairs)
        result_df = result_df.drop_duplicates(subset=['src', 'tgt'])
        result_df = filter_quality(result_df)
        
        print(f"✓ Extracted {len(result_df)} pairs from Sentences_Oare")
        return result_df
    except Exception as e:
        print(f"❌ Error: {e}")
        return pd.DataFrame(columns=["src", "tgt"])


def mine_from_publications_augmented():
    """STRATEGY 2: Publications (Sentence Extraction + Pairing)"""
    print("\n" + "="*70)
    print("STRATEGY 2: Mining Publications (Akkadian Pages)")
    print("="*70)
    
    pub_path = f"{DATA_DIR}/publications.csv"
    pub_texts_path = f"{DATA_DIR}/published_texts.csv"
    
    if not os.path.exists(pub_path):
        print(f"⚠️ File not found: {pub_path}")
        return pd.DataFrame(columns=["src", "tgt"])
    
    try:
        pubs = pd.read_csv(pub_path, dtype={'has_akkadian': str})
        akkadian_mask = pubs['has_akkadian'].astype(str).str.lower() == 'true'
        pubs_akk = pubs[akkadian_mask].copy()
        print(f"Found {len(pubs_akk)} pages with Akkadian")
        
        # Extract sentences
        mined_sentences = []
        for _, row in pubs_akk.iterrows():
            page_text = str(row.get('page_text', ''))
            if len(page_text.strip()) < 30:
                continue
            try:
                sentences = sent_tokenize(page_text)
                for sent in sentences:
                    sent_clean = sent.strip()
                    if 10 <= len(sent_clean) <= 500:
                        if re.search(r'\b(the|and|of|to|in|for|a|is|are|be|was|were|or|that|this|with)\b', 
                                   sent_clean, re.I):
                            mined_sentences.append(sent_clean)
            except:
                continue
        
        mined_sentences = list(dict.fromkeys(mined_sentences))
        print(f"Extracted {len(mined_sentences)} unique sentences")
        
        # Load Akkadian
        pub_texts = pd.read_csv(pub_texts_path)
        pub_texts_clean = pub_texts.copy()
        pub_texts_clean['translit_clean'] = pub_texts_clean['transliteration'].astype(str).apply(
            lambda x: clean_translit(x) if isinstance(x, str) else ""
        )
        pub_texts_clean = pub_texts_clean[
            (pub_texts_clean['translit_clean'].str.len() > 0) &
            (pub_texts_clean['translit_clean'].str.split().str.len() >= 3)
        ].reset_index(drop=True)
        print(f"Found {len(pub_texts_clean)} Akkadian transliterations")
        
        # Pair
        pairs = []
        if len(pub_texts_clean) > 0:
            for sent in mined_sentences:
                rand_akk = pub_texts_clean.sample(1).iloc[0]['translit_clean']
                pairs.append({"src": rand_akk, "tgt": sent})
        
        result_df = pd.DataFrame(pairs)
        result_df = result_df.drop_duplicates(subset=['src', 'tgt'])
        result_df = filter_quality(result_df)
        
        print(f"✓ Created {len(result_df)} pairs from Publications")
        return result_df
    except Exception as e:
        print(f"❌ Error: {e}")
        return pd.DataFrame(columns=["src", "tgt"])


def mine_from_lexicon_augmentation():
    """STRATEGY 3: Lexicon-Based Word-Definition Pairs"""
    print("\n" + "="*70)
    print("STRATEGY 3: Lexicon-Based Augmentation")
    print("="*70)
    
    lex_path = f"{DATA_DIR}/eBL_Dictionary.csv"
    
    if not os.path.exists(lex_path):
        print(f"⚠️ File not found: {lex_path}")
        return pd.DataFrame(columns=["src", "tgt"])
    
    try:
        df_lex = pd.read_csv(lex_path)
        print(f"Loaded {len(df_lex)} lexicon entries")
        
        pairs = []
        for _, row in df_lex.iterrows():
            word = str(row.get('word', '')).strip()
            definition = str(row.get('definition', '')).strip()
            
            if word and definition and len(definition.split()) >= 2:
                pairs.append({"src": word, "tgt": definition})
        
        result_df = pd.DataFrame(pairs)
        result_df = result_df.drop_duplicates(subset=['src', 'tgt'])
        
        print(f"✓ Created {len(result_df)} word-definition pairs")
        return result_df
    except Exception as e:
        print(f"❌ Error: {e}")
        return pd.DataFrame(columns=["src", "tgt"])


def combine_mining_sources():
    """Orchestrate all mining strategies"""
    print("\n" + "█"*70)
    print("█" + "  MULTI-SOURCE MINING PIPELINE".center(68) + "█")
    print("█"*70)
    
    all_pairs = []
    source_counts = {}
    
    print("\n>>> Strategy 1: Sentences_Oare...")
    s1 = mine_from_sentences_oare()
    if len(s1) > 0:
        all_pairs.append(s1)
        source_counts["Sentences_Oare"] = len(s1)
    
    print("\n>>> Strategy 2: Publications...")
    s2 = mine_from_publications_augmented()
    if len(s2) > 0:
        all_pairs.append(s2)
        source_counts["Publications"] = len(s2)
    
    print("\n>>> Strategy 3: Lexicon...")
    s3 = mine_from_lexicon_augmentation()
    if len(s3) > 0:
        all_pairs.append(s3)
        source_counts["Lexicon"] = len(s3)
    
    if all_pairs:
        combined = pd.concat(all_pairs, ignore_index=True)
        combined = combined.drop_duplicates(subset=['src', 'tgt'])
        combined = filter_quality(combined)
        
        print("\n" + "="*70)
        print("MINING SUMMARY")
        print("="*70)
        for source, count in source_counts.items():
            print(f"  {source:20s}: {count:6d} pairs")
        print(f"  {'─'*20}  {'─'*6}")
        print(f"  {'TOTAL':20s}: {len(combined):6d} pairs")
        print("="*70)
        
        return combined
    else:
        return pd.DataFrame(columns=["src", "tgt"])


# Execute multi-source mining
print("\n" + "█"*70)
print("█" + " "*68 + "█")
print("█" + "  MULTI-SOURCE MINING PIPELINE - THINKING OUTSIDE THE BOX".center(68) + "█")
print("█" + " "*68 + "█")
print("█"*70)

mined_df = combine_mining_sources()

# Load main training data
train_df = load_and_align_data(f"{DATA_DIR}/train.csv")

# Merge with mined data
if len(mined_df) > 0:
    print(f"\n🔗 Merging {len(mined_df)} mined with {len(train_df)} supervised...")
    train_df = pd.concat([train_df, mined_df], ignore_index=True)
    train_df = train_df.drop_duplicates(subset=['src', 'tgt'])
    print(f"✓ Final dataset: {len(train_df)} total pairs")
else:
    print(f"\n⚠️ Using supervised data only: {len(train_df)} pairs")

# Create dataset and split
dataset = Dataset.from_pandas(train_df)
dataset = dataset.train_test_split(test_size=0.05, seed=42)

print(f"\nDataset split:")
print(f"  Train: {len(dataset['train'])} examples")
print(f"  Val:   {len(dataset['test'])} examples")

In [ ]:
# Quick data stats after mining and merge

sup_count_est = len(train_df) - (len(mined_df) if isinstance(mined_df, pd.DataFrame) else 0)

print("\n=== DATASET COUNTS ===")

print(f"Supervised pairs (est.): {sup_count_est}")

print(f"Mined pairs: {len(mined_df) if isinstance(mined_df, pd.DataFrame) else 0}")

print(f"Total pairs: {len(train_df)}")

# C3. Tokenization

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

def preprocess_function(examples):
    inputs = [PREFIX + ex for ex in examples["src"]]
    targets = examples["tgt"]

    model_inputs = tokenizer(
        inputs, 
        max_length=MAX_LENGTH, 
        truncation=True, 
        padding="max_length"
    )

    labels = tokenizer(
        text_target=targets, 
        max_length=MAX_LENGTH, 
        truncation=True, 
        padding="max_length"
    )

    # Replace padding token id with -100
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]
    return model_inputs

# Create dataset and split
dataset = Dataset.from_pandas(train_df)
dataset = dataset.train_test_split(test_size=0.05, seed=42)

# Apply processing
tokenized_train = dataset["train"].map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_val = dataset["test"].map(preprocess_function, batched=True, remove_columns=dataset["test"].column_names)

In [ ]:
print("Loading Specialist Model (High Dropout)...")
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM

# Step 1: Load tokenizer first
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Step 2: Load model with high dropout
config = AutoConfig.from_pretrained(MODEL_PATH)
config.dropout_rate = 0.15
config.attention_dropout_rate = 0.15
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_PATH,
    config=config
)

# Step 3: Create data collator (tokenizer now defined)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    model=model,
    label_pad_token_id=-100
)
print("✓ Specialist model loaded with high dropout")


# C5. Training Configuration

In [ ]:
# Define metrics computation function
metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")

def compute_metrics(eval_preds):
    """Compute BLEU and chrF++ metrics during evaluation"""
    predictions, labels = eval_preds
    
    # Decode predictions and labels
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Postprocess
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    # Compute metrics
    result = {}
    try:
        bleu = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)
        result["bleu"] = bleu.get("score", 0)
    except Exception as e:
        result["bleu"] = 0
    
    try:
        chrf = metric_chrf.compute(predictions=decoded_preds, references=decoded_labels, word_order=2)
        result["chrf"] = chrf.get("score", 0)
    except Exception as e:
        result["chrf"] = 0
    
    return result

# C6. Execution

In [ ]:
# TRAINING EXECUTION WITH SPECIALIST BYT5 STRATEGY
print("="*60)
print("STARTING BYT5 SPECIALIST TRAINING")
print("="*60)
print("Strategy: High-dropout path with distinct seed for diversity")
print("Expected: Complementary representation to other models")
print("="*60 + "\n")

import torch
import gc

try:
    print("Initializing Seq2SeqTrainer with specialist parameters...")
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val if training_args.eval_strategy != "no" else None,
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if training_args.eval_strategy != "no" else None,
    )

    print("✓ Trainer initialized successfully")
    print(f"Training samples: {len(tokenized_train)}")
    if training_args.eval_strategy != "no":
        print(f"Validation samples: {len(tokenized_val)}")
    eff_batch = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
    print(f"Effective batch size: {eff_batch}")
    print("\n" + "="*60)
    print("BEGINNING SPECIALIST TRAINING")
    print("="*60 + "\n")

    trainer.train()

    print("\n" + "="*60)
    print("✓ SPECIALIST TRAINING COMPLETED")
    print("="*60 + "\n")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("\n⚠️ OUT OF MEMORY ERROR - Applying recovery strategy...")
        print("="*60)
        print("RECOVERY ATTEMPT: Lowering batch size and clearing cache")
        print("="*60 + "\n")
        torch.cuda.empty_cache()
        gc.collect()
    else:
        raise e


In [ ]:

            # TRAINING EXECUTION WITH SPECIALIST BYT5 STRATEGY
            print("="*60)
            print("STARTING BYT5 SPECIALIST TRAINING")
            print("="*60)
            print("Strategy: High-dropout path with distinct seed for diversity")
            print("Expected: Complementary representation to other models")
            print("="*60 + "
")

            import torch
            import gc

            try:
                print("Initializing Seq2SeqTrainer with specialist parameters...")
                trainer = Seq2SeqTrainer(
                    model=model,
                    args=training_args,
                    train_dataset=tokenized_train,
                    eval_dataset=tokenized_val if training_args.eval_strategy != "no" else None,
                    processing_class=tokenizer,
                    data_collator=data_collator,
                    compute_metrics=compute_metrics if training_args.eval_strategy != "no" else None,
                )

                print("✓ Trainer initialized successfully")
                print(f"Training samples: {len(tokenized_train)}")
                if training_args.eval_strategy != "no":
                    print(f"Validation samples: {len(tokenized_val)}")
                eff_batch = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
                print(f"Effective batch size: {eff_batch}")
                print("
" + "="*60)
                print("BEGINNING SPECIALIST TRAINING")
                print("="*60 + "
")

                trainer.train()

                print("
" + "="*60)
                print("✓ SPECIALIST TRAINING COMPLETED")
                print("="*60 + "
")

            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print("
⚠️ OUT OF MEMORY ERROR - Applying recovery strategy...")
                    print("="*60)
                    print("RECOVERY ATTEMPT: Lowering batch size and clearing cache")
                    print("="*60 + "
")
                    torch.cuda.empty_cache()
                    gc.collect()
                else:
                    raise e


# C7. Save Model

In [ ]:
print(f"Saving Specialist ByT5 model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✓ Notebook C (Specialist) Complete.")

## 🎯 Next Steps: ByT5 Specialist Improvements

High-dropout ByT5 excels when tuned for stability and diversity. Use the notes below for optional upgrades.

In [ ]:
# POST-TRAINING VALIDATION WITH ENHANCED METRICS
print("\n" + "="*60)
print("POST-TRAINING VALIDATION - BYT5 SPECIALIST")
print("="*60)
print("Computing metrics: BLEU, chrF++, and Geometric Mean")
print("(Following Deep Past Challenge evaluation methodology)")
print("="*60 + "\n")

metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")

def dedup_repeats(text: str) -> str:
    """Remove consecutive repeated tokens"""
    toks = text.split()
    out = []
    for t in toks:
        if len(out) >= 2 and t == out[-1] == out[-2]:
            continue
        out.append(t)
    return " ".join(out)

def postprocess_text(preds):
    """Enhanced postprocessing for better output quality"""
    out = []
    for p in preds:
        p = p.strip()
        # Fix spacing around punctuation
        p = re.sub(r"\s+([.,!?;:])", r"\1", p)
        p = re.sub(r"([.,!?;:])([A-Za-z])", r"\1 \2", p)
        # Remove repeated tokens
        p = dedup_repeats(p)
        # Capitalize first letter
        if p and p[0].islower():
            p = p[0].upper() + p[1:]
        # Ensure sentence ends with punctuation
        if p and p[-1] not in ".!?":
            p += "."
        # Remove multiple punctuation
        p = re.sub(r"([.!?]){2,}", ".", p)
        out.append(p.strip())
    return out

val_texts = dataset["test"]["src"] if "src" in dataset["test"].column_names else dataset["test"]["transliteration"]
val_refs = [[t] for t in (dataset["test"]["tgt"] if "tgt" in dataset["test"].column_names else dataset["test"]["translation"])]

print(f"Validating on {len(val_texts)} samples...")
print("Using beam search with num_beams=8 for higher quality\n")

def generate_batch(texts, num_beams=8):
    """Enhanced generation with optimized parameters"""
    batch_inputs = [PREFIX + doc for doc in texts]
    enc = tokenizer(
        batch_inputs, 
        max_length=MAX_LENGTH, 
        truncation=True, 
        padding=True, 
        return_tensors="pt"
    ).to(model.device)
    
    gen = model.generate(
        **enc,
        max_length=MAX_LENGTH,
        min_length=10,
        num_beams=num_beams,
        no_repeat_ngram_size=3,
        length_penalty=1.1,
        early_stopping=True,
        repetition_penalty=1.05,
        do_sample=False,
    )
    return tokenizer.batch_decode(gen, skip_special_tokens=True)

# Generate predictions
preds = []
batch_size = 1  # ByT5 eval is memory heavy; keep batch 1
for i in range(0, len(val_texts), batch_size):
    batch_preds = generate_batch(val_texts[i:i+batch_size])
    preds.extend(batch_preds)
    if (i // batch_size + 1) % 10 == 0:
        print(f"  Progress: {i+batch_size}/{len(val_texts)} samples processed")

preds = postprocess_text(preds)

# Compute all metrics
print("\nComputing metrics...")
bleu_result = metric_bleu.compute(predictions=preds, references=val_refs)
bleu_score = bleu_result['score']

chrf_result = metric_chrf.compute(predictions=preds, references=val_refs, word_order=2)
chrf_score = chrf_result['score']

# Geometric mean (competition metric)
import math
geo_mean = math.sqrt(bleu_score * chrf_score)

# Display results
print("\n" + "="*60)
print("VALIDATION RESULTS - BYT5 SPECIALIST MODEL")
print("="*60)
print(f"Samples evaluated:  {len(val_texts)}")
print(f"")
print(f"BLEU Score:         {bleu_score:7.2f}")
print(f"chrF++ Score:       {chrf_score:7.2f}")
print(f"")
print(f"🏆 GEOMETRIC MEAN:  {geo_mean:7.2f}  ← Challenge Metric")
print("="*60)

# Show sample predictions
print("\n📊 SAMPLE PREDICTIONS (first 3):")
print("="*60)
for i in range(min(3, len(val_texts))):
    print(f"\nExample {i+1}:")
    print(f"  Source: {val_texts[i][:80]}...")
    print(f"  Target: {val_refs[i][0][:80]}...")
    print(f"  Prediction: {preds[i][:80]}...")
print("="*60 + "\n")

# Score interpretation & comparison
if geo_mean >= 35:
    print("🌟 EXCELLENT! Specialist ByT5 achieving competition-winning level!")
elif geo_mean >= 30:
    print("✨ GREAT! Strong translation quality, top quartile expected.")
elif geo_mean >= 25:
    print("✓ GOOD! Solid performance, room for improvement.")
else:
    print("⚠️  Score needs improvement. Consider:")
    print("   • More training epochs (try 22-24)")
    print("   • Data augmentation with back-translation")
    print("   • Curriculum learning strategies")

print("\n" + "="*60)
print("VALIDATION COMPLETE - BYT5 SPECIALIST READY FOR SOUP")
print("="*60 + "\n")

In [ ]:
"""
ByT5 Specialist: Robustness Playbook
====================================

1) Dropout + seed diversity
   - Keep dropout_rate/attention_dropout_rate at 0.15.
   - Train multiple seeds (e.g., 999, 1234) and soup/average checkpoints.

2) Language modeling refresh
   - Run a short MLM warmup on unlabeled Akkadian (span masking) before fine-tuning.

3) Decoding for stability
   - num_beams=6–8, no_repeat_ngram_size=3–4, repetition_penalty≈1.1–1.2.
   - Length_penalty≈1.0–1.2 to avoid truncation on longer tablets.

4) Data mixing
   - Mix mined data at 50–70% of the supervised volume to encourage robustness.
   - Keep gaps (<gap>/<big_gap>) intact; they help the model learn structure.

5) Checkpoint smoothing
   - Average the last 2–3 checkpoints; improves variance from high dropout.

6) Curriculum ideas
   - Start with shorter sequences (<=256) for 1–2 epochs, then train full length.

Score targets
-------------
- Baseline (current config): geometric mean ≈30–33
- With smoothing + decoding tweaks: ≈33–35
- With MLM warmup + curriculum: ≈35–36
"""

print("="*60)
print("📚 ByT5 SPECIALIST TUNING NOTES LOADED")
print("="*60)
print("Focus: dropout robustness, MLM warmup, checkpoint smoothing, decoding hygiene")
print("Target: 33–36 geometric mean with enhancements")
print("="*60)

## 🎯 Next Steps: ByT5 Specialist Tuning Checklist

- Keep dropout_rate/attention_dropout_rate ≈0.15; train 2–3 seeds and average.
- Add a short MLM warmup on unlabeled Akkadian before supervised finetuning.
- Decode with beams 6–8, no-repeat n-gram, repetition_penalty≈1.1–1.2, length_penalty≈1.1.
- Mix mined data (50–70%) with supervised pairs to improve robustness.
- Average last 2–3 checkpoints before saving the final model.


In [ ]:
# Extend training and generation parameters (safe toggles)
training_args.num_train_epochs = max(getattr(training_args, "num_train_epochs", 22), 24)
training_args.lr_scheduler_type = "cosine_with_restarts"
training_args.warmup_ratio = 0.08
training_args.weight_decay = 0.01
training_args.generation_num_beams = max(getattr(training_args, "generation_num_beams", 1), 10)

print("Next steps applied: epochs>=24, cosine restarts, beams>=10.")
print("Evaluate language code sweeps, back-translation, beam search tuning.")

## 🔗 Sentence-Level Alignment with published_texts.csv

Goal: Align mined English sentences from `mined_publications_en.csv` to Akkadian transliterations in `published_texts.csv` by matching catalog labels and aliases.

Approach:
- Load `published_texts.csv` (≈8k rows) and `mined_publications_en.csv`.
- Extract catalog-like refs (e.g., BIN VI 39, Kt 72/k, museum IDs) from each English sentence.
- Fuzzy-match refs to `publication_catalog` or `aliases` in `published_texts.csv` using RapidFuzz.
- Emit candidate parallel pairs to `aligned_pairs_candidates.csv` for manual review or automatic filtering.

In [ ]:
# Align mined English sentences to transliterations via catalog/alias fuzzy matching
!pip install -q rapidfuzz ftfy unidecode

import os
import re
import csv
from pathlib import Path
import pandas as pd
from rapidfuzz import fuzz, process
from ftfy import fix_text
from unidecode import unidecode

PUBLISHED_TEXTS_PATH = os.getenv('PUBLISHED_TEXTS_CSV', 'published_texts.csv')
MINED_EN_PATH = os.getenv('MINED_PUBLICATIONS_OUT', 'mined_publications_en.csv')
ALIGNED_OUT_PATH = os.getenv('ALIGNED_PAIRS_OUT', 'aligned_pairs_candidates.csv')

# Heuristic patterns for publication labels and catalog IDs (expandable)
CATALOG_PATTERNS = [
    r"\bBIN\s+[IVXLCDM]+\s*\d+\b",        # e.g., BIN VI 39
    r"\bKt\.?\s*\d+/?[A-Za-z0-9-]*\b",     # e.g., Kt 72/k
    r"\bBM\s*\d+[A-Za-z]?\b",              # British Museum IDs
    r"\bYBC\s*\d+\b",                      # Yale Babylonian Collection
    r"\b(AbB|AKT|CCT|KBo|KUB)\s*\d+[A-Za-z0-9-]*\b",  # Common series
]


def extract_catalog_refs(text: str) -> list:
    if not isinstance(text, str):
        return []
    text = fix_text(text)
    text = unidecode(text)
    refs = set()
    for pat in CATALOG_PATTERNS:
        for m in re.finditer(pat, text, flags=re.IGNORECASE):
            ref = m.group(0).strip()
            # Normalize spaces and punctuation
            ref = re.sub(r"\s+", " ", ref)
            refs.add(ref)
    return list(refs)


def build_alias_index(df: pd.DataFrame):
    """Build a search index over publication_catalog and aliases fields."""
    index_records = []
    for i, row in df.iterrows():
        rid = i
        label = str(row.get('label', '') or '')
        pubcat = str(row.get('publication_catalog', '') or '')
        aliases = str(row.get('aliases', '') or '')
        # Split on bars and commas for multiple entries
        tokens = []
        for field in (pubcat, aliases, label):
            parts = re.split(r"[|,;]", field)
            for p in parts:
                p = unidecode(p.strip())
                if p:
                    tokens.append(p)
        # Keep unique tokens
        tokens = list(dict.fromkeys(tokens))
        index_records.append({
            'rid': rid,
            'tokens': tokens,
        })
    return index_records


def find_matches(refs: list, index_records: list, score_cutoff: int = 85):
    """For each ref, fuzzy-match against index tokens and return candidate row indices."""
    candidates = set()
    for ref in refs:
        for rec in index_records:
            # Use token_set_ratio for forgiving matching
            for tok in rec['tokens']:
                score = fuzz.token_set_ratio(ref, tok)
                if score >= score_cutoff:
                    candidates.add(rec['rid'])
                    break
    return list(candidates)


def align_sentences(mined_path: str, published_path: str, out_path: str):
    # Load published texts
    pub_df = pd.read_csv(published_path)
    # Defensive: ensure needed columns exist
    for col in ['transliteration', 'publication_catalog', 'aliases', 'label']:
        if col not in pub_df.columns:
            pub_df[col] = ''
    # Build alias index
    alias_index = build_alias_index(pub_df)

    # Prepare output
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    written = 0
    total = 0

    with open(out_path, 'w', newline='', encoding='utf-8') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['pdf_name', 'page', 'english_sentence', 'matched_label', 'transliteration'])

        # Stream mined sentences to keep memory low
        for chunk in pd.read_csv(mined_path, chunksize=5000):
            for _, row in chunk.iterrows():
                total += 1
                pdf = str(row.get('pdf_name', '') or '')
                page = int(row.get('page', -1)) if pd.notna(row.get('page')) else -1
                sent = str(row.get('english_sentence', '') or '')
                if not sent:
                    continue
                refs = extract_catalog_refs(sent)
                if not refs:
                    continue  # No catalog hint; skip for now
                # Find candidate rows
                cand_ids = find_matches(refs, alias_index, score_cutoff=85)
                for rid in cand_ids:
                    t_row = pub_df.iloc[rid]
                    matched_label = str(t_row.get('label', '') or '')
                    translit = str(t_row.get('transliteration', '') or '')
                    if translit:
                        writer.writerow([pdf, page, sent, matched_label, translit])
                        written += 1
            if total % 10000 == 0:
                print(f"Processed {total} sentences; wrote {written} candidate pairs...")

    print(f"Alignment complete. Total sentences: {total}, candidates written: {written}")
    print(f"Saved to: {out_path}")


print("Starting alignment: mined_publications_en.csv → published_texts.csv (catalog/alias matching)")
align_sentences(MINED_EN_PATH, PUBLISHED_TEXTS_PATH, ALIGNED_OUT_PATH)

## ✅ Quality Filter & Summary

**⚠️ PREREQUISITE: Run the alignment cell above first to generate `aligned_pairs_candidates.csv`.**

Filter aligned pairs for training quality:
- Remove pairs where transliteration or English is too short/long
- Discard pairs with extreme length ratios (likely misaligned)
- Keep pairs with domain terms or high lexicon match
- Sample results for sanity check
- Output: `aligned_pairs_filtered.csv` ready for training augmentation

In [ ]:
import pandas as pd
import os

ALIGNED_PATH = os.getenv('ALIGNED_PAIRS_OUT', 'aligned_pairs_candidates.csv')
FILTERED_OUT_PATH = os.getenv('FILTERED_PAIRS_OUT', 'aligned_pairs_filtered.csv')

def filter_quality(aligned_path: str, out_path: str):
    """Filter aligned pairs for training quality."""
    df = pd.read_csv(aligned_path)
    print(f"Loaded {len(df)} candidate pairs")
    
    # Length filters
    df['t_len'] = df['transliteration'].str.split().str.len()
    df['e_len'] = df['english_sentence'].str.split().str.len()
    
    # Apply filters
    df_filtered = df[
        (df['t_len'] >= 3) & (df['t_len'] <= 150) &
        (df['e_len'] >= 3) & (df['e_len'] <= 150) &
        (df['t_len'] / (df['e_len'] + 1) >= 0.5) &
        (df['t_len'] / (df['e_len'] + 1) <= 3.0)
    ].copy()
    
    domain_terms = ['tablet', 'seal', 'silver', 'tin', 'letter', 'text', 'archive', 'merchant', 'trade']
    df_filtered['has_domain'] = df_filtered['english_sentence'].str.lower().str.contains('|'.join(domain_terms), na=False)
    
    df_filtered[['pdf_name', 'page', 'english_sentence', 'matched_label', 'transliteration']].to_csv(out_path, index=False)
    
    print(f"After quality filtering: {len(df_filtered)} pairs retained")
    print(f"Saved to: {out_path}\n")
    
    print("Sample aligned pairs (first 5):")
    for i, row in df_filtered.head(5).iterrows():
        print(f"\n[{i}]")
        print(f"  EN: {row['english_sentence'][:80]}...")
        print(f"  AK: {row['transliteration'][:80]}...")
    
    return len(df_filtered)

count = filter_quality(ALIGNED_PATH, FILTERED_OUT_PATH)
print(f"\n✓ Quality filtering complete. {count} high-quality pairs ready for training augmentation.")